# Example: Batch Adaptive Sampling (2-to-2 example)
([From BET Documentation](http://ut-chg.github.io/BET/examples/example_rst_files/fromfile2D.html#fromfile2dexample))

>**Note:**
*This example shows how to generate adaptive samples in a specific way by implicitly defining an input event of interest. It does NOT show how to solve the stochastic inverse problem using these samples, which can be found by reading other examples. Thus, we only present the first few steps involved in discretizing the parameter and data spaces using a specific type of adaptive sampling. The user is referred to some other examples for filling in the remaining steps for solving the stochastic inverse problem following the construction of the adaptive samples.*

We will walk through the following [example](fromFile2D.py) that uses a linear interpolant of a 2-dimensional QoI map used to define a 2-dimensional data space. The parameter space in this example is also 2-dimensional.

This example specifically demonstrates the adaptive generation of samples using a goal-oriented adaptive sampling algorithm. This example is based upon the results shown in Section 8.5 of the manuscript [Definition and solution of a stochastic inverse problem for the Manning’s n parameter field in hydrodynamic models](http://dx.doi.org/10.1016/j.advwatres.2015.01.011) where the QoI map is given by $Q(\lambda) = (q_1(\lambda), q_6(\lambda))$. We refer the reader to that example for more information about the physical interpretation of the parameter and data space, as well as the physical locations of the observation stations defining the QoI map.

>**Note:**
>*In this example, we have used ADCIRC to generate data files based on a regular discretization of the parameter space whose sole purpose is to create an (accurate) surrogate QoI map defined as a piecewise linear interpolant. This is quite different from many of the other examples, but the use of the surrogate QoI map is immaterial. The user could also interface the sampler directly to ADCIRC, but this would require a copy of ADCIRC, the finite element mesh, and significant training on the use of this state-of-the-art shallow water equation code. The primary focus of this example is the generation of adaptive samples. If the user knows how to use the ADCIRC model, then the user may instead opt to significantly change Step (1) below to interface to ADCIRC instead of to our “model” defined in terms of the surrogate QoI map. Interfacing to ADCIRC directly would likely require the use of [PolyADCIRC](https://github.com/UT-CHG/PolyADCIRC).*

## Generating a single set of adaptive samples

### Step (0): Setting up the environment
Import the necessary modules:

In [1]:
import numpy as np
import bet.sampling.adaptiveSampling as asam
import bet.postProcess.plotDomains as pDom
import scipy.io as sio
from scipy.interpolate import griddata

### Step (1): Define the interface to the model and goal-oriented adaptive sampler
This is where we interface the adaptive sampler imported above to the model. In other examples, we have imported a Python interface to a computational model. In this example, we instead define the model as a (piecewise-defined) linear interpolant to the QoI map $Q(λ)=(q1(λ),q6(λ))Q(λ)=(q1(λ),q6(λ))$ using data read from a `.mat` [file](../matfiles/Q_2D.mat):

In [3]:
station_nums = [0, 5] # 1, 6
mdat = sio.loadmat('../matfiles/Q_2D')
Q = mdat['Q']
Q = Q[:, station_nums]
# Create experiment model
points = mdat['points']
def model(inputs):
    interp_values = np.empty((inputs.shape[0], Q.shape[1]))
    for i in xrange(Q.shape[1]):
        interp_values[:, i] = griddata(points.transpose(), Q[:, i],
            inputs)
    return interp_values

In this example, we use the adaptive sampler defined by `rhoD_kernel`, which requires an identification of a data distribution used to modify the transition kernel for input samples. The idea is to place more samples in the parameter space that correspond to a contour event of higher probability as specified by the data distribution `rho_D` shown below.

First, we create the `transition_set` with an initial step size ratio of 0.5 and a minimum, maximum step size ratio of `.5**5` and 1.0 respectively. Note that this algorithm only generates samples inside the parameter domain, `lam_domain` (see Step (2) below):

In [4]:
# Create Transition Kernel
transition_set = asam.transition_set(.5, .5**5, 1.0)

Here, we implicty designate a region of interest $\Lambda_k = Q^{-1}(D_k)$ in $\Lambda$ for some $D_k \subset \mathcal{D}$ through the use of the data distribution kernel. In this instance we choose our kernel $p_k(Q) = \rho_\mathcal{D}(Q)$, see `rhoD_kernel`.

We choose some $\lambda_{ref}$ and let $Q_{ref} = Q(\lambda_{ref})$:

In [5]:
Q_ref = mdat['Q_true']
Q_ref = Q_ref[15, station_nums] # 16th/20

We define a rectangle, $R_{ref} \subset \mathcal{D}$ centered at $Q(\lambda_{ref})$ with sides 15% the length of $q_1$ and $q_6$. Set $\rho_\mathcal{D}(q) = \dfrac{\mathbf{1}_{R_{ref}}(q)}{||\mathbf{1}_{R_{ref}}||}$:

In [6]:
bin_ratio = 0.15
bin_size = (np.max(Q, 0)-np.min(Q, 0))*bin_ratio
# Create kernel
maximum = 1/np.product(bin_size)
def rho_D(outputs):
    rho_left = np.repeat([Q_ref-.5*bin_size], outputs.shape[0], 0)
    rho_right = np.repeat([Q_ref+.5*bin_size], outputs.shape[0], 0)
    rho_left = np.all(np.greater_equal(outputs, rho_left), axis=1)
    rho_right = np.all(np.less_equal(outputs, rho_right),axis=1)
    inside = np.logical_and(rho_left, rho_right)
    max_values = np.repeat(maximum, outputs.shape[0], 0)
    return inside.astype('float64')*max_values

kernel_rD = asam.rhoD_kernel(maximum, rho_D)

The basic idea is that when the region of interest has been “found” by some sample in a chain, the transition set is modified by the adaptive sampler (it is made smaller) so that more samples are placed within this event of interest.

Given a (M, mdim) data vector `rhoD_kernel` expects that `rho_D` will return a `ndarray` of shape (M,).

Next, we create the `sampler`. This `sampler` will create 80 independent sampling chains that are each 125 samples long:

In [16]:
# Create sampler
chain_length = 125
num_chains = 80
num_samples = chain_length*num_chains
sampler = asam.sampler(num_samples, chain_length, model)
sample_save_file = 'sandbox2d'

>**Note:**
* *In the first line of code above change `chain_length` and `num_chains` to reduce the total number of forward solves.*
* *If `num_chains = 1` above, then this is no longer a “batch” sampling process where multiple chains are run simultaneously to “search for” the region of interest.*
* *Saves to `sandbox2d.mat`.*

### Step (2) [and Step (3)]: Describe and (adaptively) sample the input (and output) space
The adaptive sampling of the input space requires feedback from the corresponding output samples, so the sets of samples are, in a sense, created simultaneously in order to define the discretization of the spaces used to solve the stochastic inverse problem. While this can always be the case, in other examples, we often sampled the input space completely in one step, and then propagated the samples through the model to generate the QoI samples in another step, and these two samples sets together were used to define the discretization object used to solve the stochastic inverse problem.

The compact (bounded, finite-dimensional) paramter space for this example is:

In [17]:
lam_domain = np.array([[.07, .15], [.1, .2]])

We choose an initial sample type to seed the sampling chains, which in this case comes from using Latin-Hypercube sampling:

In [18]:
inital_sample_type = "lhs"

Finally, we adaptively generate the samples using `generalized_chains()`:

In [20]:
(my_disc,  all_step_ratios) = sampler.generalized_chains(lam_domain,
                            transition_set, kernel_rD, sample_save_file, inital_sample_type)

#### [Optional]:
We may choose to visualize the results by executing the following code:

In [21]:
# Read in points_ref and plot results
ref_sample = mdat['points_true']
ref_sample = ref_sample[5:7, 15]

# Show the samples in the parameter space
pDom.scatter_rhoD(my_disc, rho_D=rho_D, ref_sample=ref_sample, io_flag='input')
# Show the corresponding samples in the data space
pDom.scatter_rhoD(my_disc, rho_D=rho_D, ref_sample=Q_ref, io_flag='output')
# Show the data domain that corresponds with the convex hull of samples in the
# parameter space
pDom.show_data_domain_2D(my_disc, Q_ref=Q_ref)
# Show multiple data domains that correspond with the convex hull of samples in
# the parameter space
pDom.show_data_domain_multi(my_disc, Q_ref=Q_ref, showdim='all')

In [22]:
# hack to refresh html after changes within notebook
import random
__counter__ = random.randint(0,2e9)

# displays saved visualizations
from IPython.display import HTML, display
display(HTML("<table><tr><td colspan=3><center>Visualizations</center></td></tr>"+
             "<tr><td><img src='q1_q2_domain_Q_cs.png?%d'></td>"% __counter__+
             "<td><img src='rhoD_samples_cs.png?%d'></td></tr>"% __counter__+
             "</table>" ))

>**Note:**
>*The user could simply run the example [plotDomains2D.py](plotDomains2D.py) to see the results for a previously generated set of adaptive samples.*

### Steps (4)-(5) [user]: Defining and solving a stochastic inverse problem
In the call to `sampler.generalized_chains` above, a discretization object is created and saved. The user may wish to follow some of the other examples (e.g., [Example: Linear Map with Uniform Sampling](../linearMap/LinearMapExample.ipynb) or [Example: Nonlinear Map with Uniform Sampling](../nonlinearMap/NonLinearMapExample.ipynb)) along with the paper referenced above to describe a data distribution around a reference datum (Step (4)) and solve the stochastic inverse problem (Step (5)) using the adaptively generated discretization object by loading it from file. This can be done in a separate script (but do not forget to do Step (0) which sets up the environment before coding Steps (4) and (5)).

## Generating and comparing several sets of adaptive samples

In some instances the user may want to generate and compare several sets of adaptive samples using a surrogate model to determine what the best kernel, transition set, number of generalized chains, and chain length are before adaptively sampling a more computationally expensive model. See [sandbox_test_2D.py](sandbox_test_2D.py). The set up in sandbox_test_2D.py is very similar to the set up in [fromFile2D.py](fromFile2D.py), which was examined in detail above, and is omitted for brevity.

> **Note:**
>*If all of the above code in this notebook has been run, our current environment is already set up to compare several sets of adaptive samples and we can proceed with the following exploration.*

We can explore several types of kernels:

In [23]:
kernel_mm = asam.maxima_mean_kernel(np.array([Q_ref]), rho_D)
kernel_rD = asam.rhoD_kernel(maximum, rho_D)
kernel_m = asam.maxima_kernel(np.array([Q_ref]), rho_D)
kern_list = [kernel_mm, kernel_rD, kernel_m]

In [25]:
# Create sampler
chain_length = 125
num_chains = 80
num_samples = chain_length*num_chains
sampler = asam.sampler(num_samples, chain_length, model)
inital_sample_type = "lhs"

# Get samples
# Run with varying kernels
gen_results = sampler.run_gen(kern_list, rho_D, maximum, lam_domain,
        transition_set, sample_save_file)

We can explore `transition_set` with various inital, minimum, and maximum step size ratios:

In [27]:
# Run with varying transition sets bounds
init_ratio = [0.1, 0.25, 0.5]
min_ratio = [2e-3, 2e-5, 2e-8]
max_ratio = [.5, .75, 1.0]
tk_results = sampler.run_tk(init_ratio, min_ratio, max_ratio, rho_D,
        maximum, lam_domain, kernel_rD, sample_save_file)

We can explore a single kernel with varying values of ratios for increasing and decreasing the step size (i.e. the size of the hyperrectangle to draw a new step from using a transition set):

In [33]:
increase = [1.0, 2.0, 4.0]
decrease = [0.5, 0.5e2, 0.5e3]
tolerance = [1e-4, 1e-6, 1e-8]
incdec_results = sampler.run_inc_dec(increase, decrease, tolerance, rho_D,
        maximum, lam_domain, transition_set, sample_save_file)

>**Note**: *The above examples just use a zip combination of the lists uses to define varying parameters for the kernels and transition sets. To explore the product of these lists you need to use numpy.meshgrid and numpy.ravel or a similar process.*

>**Note:** *To compare the results in terms of yield or the total number of samples generated in the region of interest we could use `compare_yield` to display the results to screen, however this tool is being depreciated. Here compare_yield() simply would display to screen the `sample_quality` and `run_param` sorted by `sample_quality` and indexed by `sort_ind`.*

In [44]:
import bet.postProcess.postTools as ptools
# Compare the quality of several sets of samples
print "Compare yield of sample sets with various kernels"
ptools.compare_yield(gen_results[3], gen_results[2], gen_results[4])
print "Compare yield of sample sets with various transition sets bounds"
ptools.compare_yield(tk_results[3], tk_results[2], tk_results[4])
print "Compare yield of sample sets with variouos increase/decrease ratios"
ptools.compare_yield(incdec_results[3], incdec_results[2],incdec_results[4])


Compare yield of sample sets with various kernels


PendingDeprecationWarning: 